In [12]:
import sys
import os
import openai
sys.path.append('../..')
#!pip install langchain

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv('C:/Users/alexc/.ipynb_checkpoints/rag_project/.env') # read local .env file
open.api_key = os.environ['OPENAI_API_KEY']

In [37]:
df = pd.read_csv('C:/Users/alexc/multmodal_agent/relacionamentos_jn/relacionamentos.csv')

In [38]:
df.head()

,id,titulo,disciplina,ano,status,id_anterior,titulo_anterior,disciplina_anterior,ano_anterior,status_anterior
0,5,Organizando nomes próprios,L. Portuguesa,1,1,20629,Você realmente conhece o alfabeto?,L. Portuguesa,1,1
1,5,Organizando nomes próprios,L. Portuguesa,1,1,10018,As vogais,L. Portuguesa,1,1
2,5,Organizando nomes próprios,L. Portuguesa,1,1,10037,As consoantes,L. Portuguesa,1,1
3,6,Receitas e outras listas contextualizadas,L. Portuguesa,1,1,106,Jogos com regras,L. Portuguesa,1,1
4,9,Hipo e Hipersegmentação,L. Portuguesa,2,1,10025,Estrutura das sílabas,L. Portuguesa,2,1


In [62]:
import pandas as pd
from io import StringIO
from langchain.prompts import PromptTemplate
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI

# Substitua "sua_chave_api_aqui" pela sua chave de API da OpenAI
openai_api_key = "sk-proj-nGphcyAtAeWW0RIXF3VFT3BlbkFJo7uOiTu5GT57lch8Lt2E"

# Carrega o arquivo CSV
file_path = 'C:/Users/alexc/multmodal_agent/relacionamentos_jn/relacionamentos.csv'
df = pd.read_csv(file_path)

# Converte os dados filtrados para uma string CSV
content_data = df[['titulo', 'disciplina', 'ano', 'titulo_anterior']].to_csv(index=False)

# Define o modelo de linguagem e o template de prompt para verificar o conteúdo
check_content_prompt = PromptTemplate(
    input_variables=["content_data", "question"],
    template="""
Você é um especialista educacional que conhece a BNCC brasileira. Você receberá uma lista de conteúdos das disciplinas, separados por ano escolar, juntamente com os conteúdos prévios necessários para cada título. Sua tarefa é verificar se os conteúdos listados estão corretos, se há mais conteúdos a serem considerados que não estão listados, sugerir melhorias e reorganizações na lista, tudo baseado na BNCC brasileira.

Aqui está a lista de conteúdos:

{content_data}

{question}

Instruções:
1. Verifique se os conteúdos listados estão corretos e em conformidade com a BNCC.
2. Adicione conteúdos adicionais importantes para o ano escolar que deveriam estar presentes, mas estão faltando.
3. Recomende reorganizações na lista, quando necessárias.
4. A coluna 'titulo_anterior' se refere a aprendizagens anteriores consideradas essenciais para o domínio dos conteúdos da coluna 'título'. 
   Verifique se ela está correta e adequada, ou seja, faltam conteúdos prévios importantes e que não estão no documento.
5. Se há conteúdos faltantes importantes na coluna 'titulo_anterior', busque conteúdos presentes na lista e que são importantes para o domínio do conhecimento referido na coluna 'titulo' da respectiva habilidade e adicione no novo documento.
6. Como resultado das análises dos itens 5 e 6, produza um novo documento contendo todas as sugestões de conhecimentos prévios a serem inseridos na coluna 'titulo_anterior', referentes aos conteúdos presente na coluna 'título' e que não estão contemplados no documento.
7. Os conteúdos da coluna 'titulo_anterior' são essenciais para manter o fluxo de aprendizagem dos alunos. Eles representam os conhecimentos prévios exigidos para cada conhecimento representado em 'título'. Analise com base na BNCC brasileira e faça todas as sugestões de adição de novos conteúdos para a coluna 'titulo_anterior'.
8. Você deve se ater apenas aos conteúdos presentes no documento para realizar as sugestões de alteração.
9. Ao analisar um conteúdo de matemática, busque todos os conteúdos de matemática do documento, de todos os anos, e analise se há conteúdos que deveriam estar na coluna 'titulo_anterior' mas não estão. O critério é o seguinte: há conhecimentos prévios para os alunos que são importantes, estão presentes na lista, mas não fazem parte deste conhecimento.
10. A sua resposta deve ser um documento tal qual {content_data}, contendo apenas a sua resposta para a {question} com as mesmas colunas, sendo que a coluna'titulo_anterior' deve
ser editada, com os acréscimos de conteúdo que você sugere
11. Responda apenas o que for perguntado. Devolva na saída apenas a resposta da disciplina e ano constante na question
"""
)

# Inicializa o modelo de linguagem com a chave API e define a cadeia de prompts
llm = ChatOpenAI(api_key=openai_api_key, model="gpt-4-1106-preview", temperature=0.3)
check_content_chain = LLMChain(llm=llm, prompt=check_content_prompt)

# Define a pergunta para o modelo
question = "Quais conteúdos adicionais de Matemática do 5º ano deveriam ser incluídos? Os conhecimentos da coluna titulo_anterior estão adequados?"

# Prepara a entrada do prompt
prompt_input = {"content_data": content_data, "question": question}

# Executa a cadeia de prompts para análise
response = check_content_chain.run(prompt_input)

# Exibe a resposta do modelo
print(response)

# Verifica se a resposta do modelo é um CSV bem formatado antes de tentar ler
def is_valid_csv(data: str) -> bool:
    try:
        pd.read_csv(StringIO(data))
        return True
    except pd.errors.ParserError:
        return False

# Salva o novo CSV se a resposta for válida
if is_valid_csv(response):
    # Converte a resposta em um DataFrame
    response_data = pd.read_csv(StringIO(response))

    # Salva o DataFrame como um novo arquivo CSV
    output_file_path = f'C:/Users/alexc/multmodal_agent/relacionamentos_jn/matematica_5.csv'
    response_data.to_csv(output_file_path, index=False)

    print("Arquivo atualizado salvo com sucesso.")
else:
    print("A resposta do modelo não é um CSV válido.")

# Para debug: se a resposta não for válida, salve-a em um arquivo de texto para inspeção
if not is_valid_csv(response):
    with open('response_debug.txt', 'w', encoding='utf-8') as f:
        f.write(response)
    print("Resposta do modelo salva para inspeção.")


titulo,disciplina,ano,titulo_anterior
Porcentagem,Matemática,5,Multiplicação e divisão de números racionais
Porcentagem,Matemática,5,Divisão exata e não exata
Porcentagem,Matemática,5,Multiplicação com dois algarismos
Porcentagem,Matemática,5,Adição e subtração de números naturais e racionais
Fração de uma quantidade,Matemática,5,Frações Unitárias
Fração de uma quantidade,Matemática,5,Representações fracionária e decimal
Representação decimal de números fracionários.,Matemática,5,Fração de uma quantidade
Representação decimal de números fracionários.,Matemática,5,Representação fracionária dos números racionais
Representação decimal de números fracionários.,Matemática,5,Multiplicação e divisão de números racionais
Princípio multiplicativo de contagem,Matemática,5,Multiplicação e divisão de números racionais
Princípio multiplicativo de contagem,Matemática,5,Cálculo mental
Princípio multiplicativo de contagem,Matemática,5,Multiplicação com dois algarismos
Localização e coordenadas,Matemát

In [63]:
csv_file_path = 'C:/Users/alexc/multmodal_agent/relacionamentos_jn/matematica_5.csv'
df = pd.read_csv(csv_file_path)

# Define o caminho do arquivo Excel de saída
excel_file_path = 'C:/Users/alexc/multmodal_agent/relacionamentos_jn/matematica_5.xlsx'

# Salva o DataFrame como um arquivo Excel
df.to_excel(excel_file_path, index=False)

print(f"Arquivo Excel salvo com sucesso em: {excel_file_path}")

Arquivo Excel salvo com sucesso em: C:/Users/alexc/multmodal_agent/relacionamentos_jn/matematica_5.xlsx


In [ ]:
from flask import Flask, render_template, request, send_file
import pandas as pd
from io import StringIO
from langchain.prompts import PromptTemplate
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
import os

app = Flask(__name__)

openai_api_key = "sk-proj-nGphcyAtAeWW0RIXF3VFT3BlbkFJo7uOiTu5GT57lch8Lt2E"

llm = ChatOpenAI(api_key=openai_api_key, model="gpt-4-1106-preview", temperature=0.3)

check_content_prompt = PromptTemplate(
    input_variables=["content_data", "question"],
    template="""
    Você é um especialista educacional que conhece a BNCC brasileira. Você receberá uma lista de conteúdos das disciplinas, separados por ano escolar, juntamente com os conteúdos prévios necessários para cada título. Sua tarefa é verificar se os conteúdos listados estão corretos, se há mais conteúdos a serem considerados que não estão listados, sugerir melhorias e reorganizações na lista, tudo baseado na BNCC brasileira.

    Aqui está a lista de conteúdos:

    {content_data}

    {question}

    Instruções:
    1. Verifique se os conteúdos listados estão corretos e em conformidade com a BNCC.
    2. Adicione conteúdos adicionais importantes para o ano escolar que deveriam estar presentes, mas estão faltando.
    3. Recomende reorganizações na lista, quando necessárias.
    4. A coluna 'titulo_anterior' se refere a aprendizagens anteriores consideradas essenciais para o domínio dos conteúdos da 
       coluna 'título'. 
       Verifique se ela está correta e adequada, ou seja, faltam conteúdos prévios importantes e que não estão no documento.
    5. Se há conteúdos faltantes importantes na coluna 'titulo_anterior', busque conteúdos presentes na lista e que são 
       importantes para o domínio do conhecimento referido na coluna 'titulo' da respectiva habilidade e adicione no novo documento.
    6. Como resultado das análises dos itens 5 e 6, produza um novo documento contendo todas as sugestões de conhecimentos prévios a 
       serem inseridos na coluna 'titulo_anterior', referentes aos conteúdos presente na coluna 'título' e que não estão contemplados no documento.
    7. Os conteúdos da coluna 'titulo_anterior' são essenciais para manter o fluxo de aprendizagem dos alunos. Eles representam os 
       conhecimentos prévios exigidos para cada conhecimento representado em 'título'. Analise com base na BNCC brasileira e faça todas as sugestões de adição de novos conteúdos para a coluna 'titulo_anterior'.
    8. Você deve se ater apenas aos conteúdos presentes no documento para realizar as sugestões de alteração.
    9. Ao analisar um conteúdo de matemática, busque todos os conteúdos de matemática do documento, de todos os anos, e analise se
       há conteúdos que deveriam estar na coluna 'titulo_anterior' mas não estão. O critério é o seguinte: há conhecimentos prévios 
       para os alunos que são importantes, estão presentes na lista, mas não fazem parte deste conhecimento.
    """
)

check_content_chain = LLMChain(llm=llm, prompt=check_content_prompt)

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        disciplina = request.form['disciplina']
        ano = request.form['ano']
        output_path = request.form['output_path']
        
        file_path = 'C:/Users/alexc/multmodal_agent/relacionamentos_jn/relacionamentos.csv'
        df = pd.read_csv(file_path)

        content_data = df[['titulo', 'disciplina', 'ano', 'titulo_anterior']].to_csv(index=False)
        question = f"Quais conteúdos adicionais de {disciplina} do {ano} deveriam ser incluídos? Os conhecimentos da coluna titulo_anterior estão adequados?"

        prompt_input = {"content_data": content_data, "question": question}
        response = check_content_chain.run(prompt_input)

        if is_valid_csv(response):
            response_data = pd.read_csv(StringIO(response))
            response_data.to_excel(output_path, index=False)
            return send_file(output_path, as_attachment=True)
        else:
            return "A resposta do modelo não é um CSV válido."

    return render_template('index.html')

def is_valid_csv(data: str) -> bool:
    try:
        pd.read_csv(StringIO(data))
        return True
    except pd.errors.ParserError:
        return False

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.13:5000
Press CTRL+C to quit
